### 1 - Define Functions

In [14]:
import sys
print(sys.executable)

c:\Users\Carst\OneDrive\Dokumente\VSC\TradingBot\.venv\Scripts\python.exe


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
print("API Key:", os.getenv("CAPITAL_API_KEY"))
print("Base URL:", os.getenv("CAPITAL_API_BASE"))


API Key: dein_key
Base URL: https://api-capital.backend-capital.com


In [4]:
# ZELLE 2 — Capital.com WebSocket: verbinden & Ticks abonnieren (nur lesen)

import json, asyncio, websockets, nest_asyncio
nest_asyncio.apply()

# 1–2 Instrumente für den Start
INSTRUMENTS = ["CS.D.BITCOIN.CFD.IP", "CS.D.ETHUSD.CFD.IP"]

# gleicher Stream-Host wie in deinem VB-Projekt:
STREAM_HOST = "wss://api-streaming-capital.backend-capital.com/connect"

# Tokens stammen aus Zelle 1:
assert CST and XSEC, "Bitte zuerst Zelle 1 (Login) ausführen – CST/XSEC fehlen."
ws_url = f"{STREAM_HOST}?CST={CST}&X-SECURITY-TOKEN={XSEC}"

# Minimal: nur Ticks (marketData). Kerzen können wir danach zuschalten.
subscribe_prices = {
    "destination": "marketData.subscribe",
    "correlationId": "1",
    "cst": CST,
    "securityToken": XSEC,
    "payload": {"epics": INSTRUMENTS},
}

# optionaler Keepalive
ping_msg = {"destination": "ping", "correlationId": "999", "cst": CST, "securityToken": XSEC}

async def run_stream():
    print("Verbinde zu:", ws_url)
    async with websockets.connect(ws_url) as ws:
        await ws.send(json.dumps(subscribe_prices))
        print("Subscribed:", INSTRUMENTS)

        # 10 Nachrichten lesen und anzeigen
        for i in range(10):
            msg = await ws.recv()
            print(f"[{i+1}] {msg}")

        # optional: Ping senden
        await ws.send(json.dumps(ping_msg))
        print("Ping gesendet – fertig.")

await run_stream()


NameError: name 'CST' is not defined

In [15]:
import asyncio
import websockets
import nest_asyncio

# erlaubt asyncio in Jupyter
nest_asyncio.apply()

async def ws_echo_test():
    url = "wss://echo.websocket.events"  # öffentlicher Test-Endpunkt
    print("Verbinde zu:", url)
    async with websockets.connect(url) as ws:
        # Server sendet oft sofort eine Begrüßungs-/Status-Nachricht
        try:
            hello = await asyncio.wait_for(ws.recv(), timeout=3)
            print("Empfangen (Server):", hello)
        except asyncio.TimeoutError:
            print("Keine Begrüßungsnachricht (ok).")

        # eigene Nachricht schicken und Antwort lesen
        await ws.send("hello tradingbot")
        reply = await asyncio.wait_for(ws.recv(), timeout=5)
        print("Echo-Antwort:", reply)

        # optional: Ping/Pong
        pong_waiter = await ws.ping()
        await asyncio.wait_for(pong_waiter, timeout=3)
        print("Ping/Pong ok.")

asyncio.run(ws_echo_test())


Verbinde zu: wss://echo.websocket.events


gaierror: [Errno 11001] getaddrinfo failed

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()
print("API Key:", os.getenv("CAPITAL_API_KEY"))
print("User:", os.getenv("CAPITAL_USERNAME"))


API Key: dein_key
User: dein_username


In [ ]:
import os, requests
from dotenv import load_dotenv
load_dotenv()

BASE = os.getenv("CAPITAL_API_BASE", "https://api-capital.backend-capital.com")
API_KEY = os.getenv("CAPITAL_API_KEY")
USER   = os.getenv("CAPITAL_USERNAME")
PWD    = os.getenv("CAPITAL_PASSWORD")

headers = {"X-CAP-API-KEY": API_KEY, "Content-Type": "application/json", "Accept": "application/json"}
payload = {"identifier": USER, "password": PWD, "encryptedPassword": False}

r = requests.post(f"{BASE}/api/v1/session", headers=headers, json=payload)
print("HTTP:", r.status_code, r.reason, r.text[:120])

CST  = r.headers.get("CST")
XSEC = r.headers.get("X-SECURITY-TOKEN")
print("CST vorhanden?", bool(CST), "XSEC vorhanden?", bool(XSEC))


HTTP: 200 
CST: 2OuJzxrEYqKFJLf9BdV7N9VB
X-SECURITY-TOKEN: iwNml4z0g7pOoLYvdse9XHMcBahmb3E


In [7]:
import pandas as pd
import pandas_ta as ta
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

ModuleNotFoundError: No module named 'pandas_ta'

In [ ]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

In [ ]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0

In [2]:
# ZELLE 1 — Capital.com Login (REST): holt CST/XSEC
import os, requests
from dotenv import load_dotenv

load_dotenv()

BASE = os.getenv("CAPITAL_API_BASE", "https://api-capital.backend-capital.com")
API_KEY = os.getenv("CAPITAL_API_KEY")
USER   = os.getenv("CAPITAL_USERNAME")
PWD    = os.getenv("CAPITAL_PASSWORD")

assert API_KEY and USER and PWD, "Bitte .env mit CAPITAL_API_KEY / CAPITAL_USERNAME / CAPITAL_PASSWORD füllen."

headers = {"X-CAP-API-KEY": API_KEY, "Content-Type": "application/json", "Accept": "application/json"}
payload = {"identifier": USER, "password": PWD, "encryptedPassword": False}

r = requests.post(f"{BASE}/api/v1/session", headers=headers, json=payload, timeout=15)
print("HTTP:", r.status_code, r.reason)
if r.status_code != 200:
    print("Antwort:", r.text)
    raise SystemExit("Login fehlgeschlagen.")

CST  = r.headers.get("CST")
XSEC = r.headers.get("X-SECURITY-TOKEN")
print("CST vorhanden?", bool(CST), "XSEC vorhanden?", bool(XSEC))


HTTP: 200 
CST vorhanden? True XSEC vorhanden? True


In [4]:
# ZELLE 2 — WebSocket: verbinden & Ticks lesen (nur anzeigen, keine Orders!)
import json, asyncio, websockets, nest_asyncio
nest_asyncio.apply()

# EPICs (nicht BTCUSD/ETHUSD-Kürzel!)
INSTRUMENTS = ["BTCUSD", "ETHUSD"]

STREAM_HOST = "wss://api-streaming-capital.backend-capital.com/connect"
assert CST and XSEC, "Bitte zuerst ZELLE 1 ausführen – CST/XSEC fehlen."

ws_url = f"{STREAM_HOST}?CST={CST}&X-SECURITY-TOKEN={XSEC}"
subscribe_prices = {
    "destination": "marketData.subscribe",
    "correlationId": "1",
    "cst": CST,
    "securityToken": XSEC,
    "payload": {"epics": INSTRUMENTS},
}

async def run_stream():
    print("Verbinde zu:", ws_url)
    async with websockets.connect(ws_url) as ws:
        await ws.send(json.dumps(subscribe_prices))
        print("Subscribed:", INSTRUMENTS)
        # 10 Nachrichten lesen
        for i in range(10):
            msg = await ws.recv()
            print(f"[{i+1}] {msg}")

await run_stream()


Verbinde zu: wss://api-streaming-capital.backend-capital.com/connect?CST=A9iyToTSI4fMCtCEaFR2TJm5&X-SECURITY-TOKEN=LH4RAPHqkd31sapB4TxcS6QT9Rv8Hau
Subscribed: ['BTCUSD', 'ETHUSD']
[1] {"status":"ERROR","destination":"marketData.subscribe","correlationId":"1","payload":{"errorCode":"error.invalid.session.token"}}


CancelledError: 

In [5]:
import os, requests
BASE = os.getenv("CAPITAL_API_BASE")
headers = {"X-CAP-API-KEY": os.getenv("CAPITAL_API_KEY"), "Accept": "application/json",
           "CST": CST, "X-SECURITY-TOKEN": XSEC}
r = requests.get(f"{BASE}/api/v1/markets?search=BTC", headers=headers, timeout=15)
print(r.status_code)
print(r.text[:800])


401
{"errorCode":"error.invalid.session.token"}


### 2 - Connect to the market and execute trades

In [ ]:
access_token="jfdkslfj" #you need token here generated from OANDA account
accountID="456dsa456"

In [ ]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M5)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)
    print(candle)


True
<Candle: 2024-01-24 02:55 PM>
True
<Candle: 2024-01-24 03 PM>
True
<Candle: 2024-01-24 03:05 PM>


In [ ]:
def count_opened_trades():
    client = API(access_token=access_token)
    r = trades.OpenTrades(accountID=accountID)
    client.request(r)
    return len(r.response['trades'])

In [ ]:
def get_candles_frame(n):
    candles = get_candles(n)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    dfstream["ATR"] = ta.atr(dfstream.High, dfstream.Low, dfstream.Close, length=7)
    dfstream["EMA_fast"]=ta.ema(dfstream.Close, length=30)
    dfstream["EMA_slow"]=ta.ema(dfstream.Close, length=50)
    dfstream['RSI']=ta.rsi(dfstream.Close, length=10)
    my_bbands = ta.bbands(dfstream.Close, length=15, std=1.5)
    dfstream=dfstream.join(my_bbands)

    return dfstream    

In [ ]:
import sys
from datetime import datetime

def trading_job():

    dfstream = get_candles_frame(70)

    signal = total_signal(dfstream, len(dfstream)-1, 7) # previous candle signal current candle still opened
        
    slatr = 1.1*dfstream.ATR.iloc[-1]
    TPSLRatio = 1.5
    max_spread = 16e-5
    
    candle = get_candles(1)[-1]
    candle_open_bid = float(str(candle.bid.o))
    candle_open_ask = float(str(candle.ask.o))
    spread = candle_open_ask-candle_open_bid

    SLBuy = candle_open_bid-slatr-spread
    SLSell = candle_open_ask+slatr+spread

    TPBuy = candle_open_ask+slatr*TPSLRatio+spread
    TPSell = candle_open_bid-slatr*TPSLRatio-spread
    
    client = API(access_token=access_token)
    #Sell
    if signal == 1 and count_opened_trades() == 0 and spread<max_spread:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-3000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2 and count_opened_trades() == 0 and spread<max_spread:
        mo = MarketOrderRequest(instrument="EUR_USD", units=3000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 3 - Executing orders automatically with a scheduler

In [ ]:
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56',
                  start_date='2023-12-08 12:00:00', timezone='America/Chicago')
scheduler.start()

Job "trading_job (trigger: cron[day_of_week='mon-fri', hour='0-23', minute='1,6,11,16,21,26,31,36,41,46,51,56'], next run at: 2024-01-24 09:16:00 CST)" raised an exception
Traceback (most recent call last):
  File "c:\CodeLab\Python\Lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\710177\AppData\Local\Temp\ipykernel_11920\80317498.py", line 25, in trading_job
    logging.info("{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14}".format(
    ^^^^^^^
NameError: name 'logging' is not defined
